In [ ]:
## Duplicate of finding_award_winners

In [1]:
import numpy as np
import pandas as pd
import spacy
from spacy import displacy
from collections import defaultdict
import en_core_web_md
nlp = en_core_web_md.load()

In [2]:
award_names = ['Best Motion Picture – Drama', 'Best Motion Picture – Musical or Comedy', 'Best Motion Picture – Foreign Language',
 'Best Motion Picture – Animated', 'Best Director – Motion Picture', 'Best Actor in a Motion Picture – Drama',
 'Best Actor in a Motion Picture – Musical or Comedy', 'Best Actress in a Motion Picture – Drama',
 'Best Actress in a Motion Picture – Musical or Comedy', 'Best Supporting Actor – Motion Picture',
 'Best Supporting Actress – Motion Picture', 'Best Screenplay – Motion Picture', 'Best Original Score – Motion Picture',
 'Best Original Song – Motion Picture', 'Best Television Series – Drama', 'Best Television Series – Musical or Comedy',
 'Best Miniseries or Motion Picture – Television', 'Best Actor in a Television Series – Drama',
 'Best Actor in a Television Series – Musical or Comedy', 'Best Actor in a Miniseries or Motion Picture – Television',
 'Best Actress in a Television Series – Drama', 'Best Actress in a Television Series – Musical or Comedy',
 'Best Actress in a Miniseries or Motion Picture – Television',
 'Best Supporting Actor in a Series, Miniseries or Motion Picture – Television',
 'Best Supporting Actress in a Series, Miniseries or Motion Picture – Television']

new_award_names = [0]*len(award_names)
for i in range(len(award_names)):
#     new_award_names[i] = award_names[i].lower().
    new_award_names[i] = award_names[i].replace(" in ", " ").replace(" a ", " ").replace(" or ", " ").replace(" – ", " ").replace(" - ", " ").lower()
#     if "television" in new_award_names[i]:
#         new_award_names[i].append("tv")
new_award_names

['best motion picture drama',
 'best motion picture musical comedy',
 'best motion picture foreign language',
 'best motion picture animated',
 'best director motion picture',
 'best actor motion picture drama',
 'best actor motion picture musical comedy',
 'best actress motion picture drama',
 'best actress motion picture musical comedy',
 'best supporting actor motion picture',
 'best supporting actress motion picture',
 'best screenplay motion picture',
 'best original score motion picture',
 'best original song motion picture',
 'best television series drama',
 'best television series musical comedy',
 'best miniseries motion picture television',
 'best actor television series drama',
 'best actor television series musical comedy',
 'best actor miniseries motion picture television',
 'best actress television series drama',
 'best actress television series musical comedy',
 'best actress miniseries motion picture television',
 'best supporting actor series, miniseries motion picture

In [36]:
awardnominees = {}
awardnominees = defaultdict(lambda: 1, awardnominees)

for award in award_names:
    awardnominees[award] = []

df = pd.read_json('../gg2013.json')
df = df['text']
df = df.str.replace('#GoldenGlobes|golden|globes|globe', "", case = False)
# nominees_df = df[df.str.contains('nominated', case = False)]
nominees_df = df[df.str.contains('Best', case = True)]
# nominees_df = nominees_df[~nominees_df.str.contains('win|RT', case = False)]
# nominees_df = nominees_df[nominees_df.str.contains('nominated', case = False)]
# df = df[~df.str.contains('think|thinking|should|maybe', case = False)]

best_director_df = df[df.str.contains('best director', case = False)]

if nominees_df.shape[0] > 8000:
    nominees_df = nominees_df.sample(n=8000)
nominees_df.shape[0]

awardnominees

<ipython-input-36-f7e1efd55735>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df = df.str.replace('#GoldenGlobes|golden|globes|globe', "", case = False)


defaultdict(<function __main__.<lambda>()>,
            {'Best Motion Picture – Drama': [],
             'Best Motion Picture – Musical or Comedy': [],
             'Best Motion Picture – Foreign Language': [],
             'Best Motion Picture – Animated': [],
             'Best Director – Motion Picture': [],
             'Best Actor in a Motion Picture – Drama': [],
             'Best Actor in a Motion Picture – Musical or Comedy': [],
             'Best Actress in a Motion Picture – Drama': [],
             'Best Actress in a Motion Picture – Musical or Comedy': [],
             'Best Supporting Actor – Motion Picture': [],
             'Best Supporting Actress – Motion Picture': [],
             'Best Screenplay – Motion Picture': [],
             'Best Original Score – Motion Picture': [],
             'Best Original Song – Motion Picture': [],
             'Best Television Series – Drama': [],
             'Best Television Series – Musical or Comedy': [],
             'Best Mini

In [13]:
def closeenough(a,b): 
  a = a.split() 
  b = b.split() 
  k = set(a).symmetric_difference(set(b)) 
  return True if len(k) <= 2 else False

In [14]:
from spacy.matcher import Matcher
from spacy.tokens import Span
from string import punctuation
import operator

In [37]:
for i, text in nominees_df.iteritems():
    for i, award in enumerate(new_award_names):
        awardlst = award.split()
        awardlst2 = award.replace("television ", "").split()
        if all(x in text.lower() for x in awardlst) or all(x in text.lower() for x in awardlst2):
            for ent in nlp(text).ents:
                if (ent.label_ == "PERSON" or ent.label == "WORK_OF_ART") and '#' not in ent.text and '@' not in ent.text:
                    awardnominees[list(awardnominees)[i]].append(ent.text)
                    
awardnominees
        

defaultdict(<function __main__.<lambda>()>,
            {'Best Motion Picture – Drama': ['Gw salah di noms ini RT',
              'Jessica Chastain - Zero',
              'Daniel Day-Lewis',
              'Daniel Day-Lewis',
              'Affleck',
              'Jessica Chastain - Zero',
              'Daniel Day Lewis',
              'Jessica Chastain',
              'Lewis - Lincoln',
              'Ben',
              'Jessica Chastain',
              'Daniel Day-Lewis',
              'George Clooney',
              'Ben Affleck',
              'George Clooney',
              'Ben Affleck',
              'Lewis - Lincoln',
              'Daniel Day-Lewis',
              'Motion Picture - Drama',
              'George Clooney',
              'Ben Affleck',
              'Lewis - Lincoln',
              'Lewis Lincoln',
              'Daniel Day Lewis',
              'Lewis - Lincoln',
              'Jessica Chastain',
              'Daniel Day-Lewis',
              'Jessica Chastai

In [6]:
pattern1 = [{"ORTH":'Best'}, {"DEP": 'compound', 'OP':'+'}, {"POS":'NOUN', 'OP':'*'}, {'IS_PUNCT': True, 'OP':'?'}, {"POS": 'PROPN', 'OP':'*'}]
pattern2 = [{"ORTH":'Best'}, {"ENT_TYPE": "WORK_OF_ART", 'OP':'+'}]


In [7]:
matcher = Matcher(nlp.vocab)
matcher.add("Award", [pattern1])

In [8]:
real = {}
real = defaultdict(lambda: 1, real)
maybe = {}
maybe = defaultdict(lambda: 1, maybe)
maybenot = {}
maybenot = defaultdict(lambda: 1, maybenot)
for i, text in nominees_df.iteritems():
    doc = nlp(text)
    matches = matcher(doc)
    if len(matches) != 0:
        span = doc[matches[-1][1]:matches[-1][2]]
        sent = str(span).strip(punctuation).strip().replace('- ','').lower()
        if "tv" in sent:
            sent.replace("tv",'television')
            
        print(sent)
            
        awarded = 0
        for i, award in enumerate(new_award_names):
            if sent == award or sent in award or closeenough(sent, award) or award in sent:
                for ent in nlp(text.replace("sent","")).ents:
                    if ent.label_ == "PERSON" or ent.label_ == "WORK_OF_ART":
                        if "Best" not in ent.text:
                            awardnominees[list(awardnominees)[i]].append(ent.text)
                            awarded = 1
            if awarded == 1:
                break
        
        if awarded == 0:
            maybe[sent] += 1
        

best drama film department
best supporting actress
best original song
best motion picture
best original song
best supporting actor
best tv series comedy
best original score
best original screenplay
best original song
best mini-series
best motion picture drama
best movie actor
best motion picture musical comedy
best tv series
best motion picture (comedy
best motion picture
best tv actor, comedy
best animated feature film brave
best supporting actor
best motion picture comedy
best supporting actress
best original score mychael danna
best awards moment
best supporting actress
best actor motion picture drama
best supporting actress
best drama movie
best supporting actor
best motion picture drama
best supporting actress
best director award
best supporting actress
best supporting actor
best actor drama
best tv series
best original song
best television series comedy
best supporting actr
best actor tv miniseries costner
best supporting actress
best tv series
best original song
best motion pict

best supporting actor
best motion picture drama
best screenplay quentin tarantino
best actor
best supporting actress
best acceptance speech
best supporting actor winner
best supporting actress
best original song: adele
best original song
best motion picture drama
best supporting actress
best foreign film amour
best original score mychael danna
best supporting actress
best motion picture comedy
best motion picture comedy
best original song: adele
best motion picture comedy
best foreign film amour
best tv series actress comedy
best actor winner
best supporting actress
best supporting actor
best tv series
best supporting actress tv
best foreign film amour
best foreign film
best original score
best supporting actress
best original song
best motion picture comedy
best director motion picture
best motion picture
best television series
best television drama
best animated feature film brave
best screenplay win
best supporting actress
best supporting actress
best supporting actress
best origina

best motion picture
best original song
best comedy tv series
best original song http://t.co/6ica6qqm
best television series
best supporting actress
best bond song
best director award
best tv series drama
best motion picture
best motion picture comedy
best supporting actor
best tv series actress comedy
best original song skyfall
best supporting actor
best supporting actress
best tv series drama
best supporting actress
best director motion picture
best picture
best supporting actor
best original score
best motion picture comedy
best motion picture drama
best motion picture comedy
best red carpet quotes
best drama series
best tv series
best original song
best movie actress
best motion picture comedy
best television series
best animated feature film brave
best original song
best supporting actress
best screenplay quentin tarantino
best screenplay quentin tarantino
best actress award
best animated feature film brave
best motion picture comedy
best supporting actor
best supporting actress
be

best original song
best supporting actress
best supporting actor
best supporting actress
best supporting actress
best motion picture
best motion picture drama
best supporting actor
best supporting actress
best television series comedy
best supporting actress
best actress tv series comedy
best original score mychael danna
best motion picture comedy
best tv series
best original song
best motion picture comedy
best director award
best original song
best motion picture, musical
best motion picture drama
best motion picture
best director oscar
best motion picture comedy
best actor motion picture comedy
best original song
best original song
best dir award
best motion picture comedy
best supporting actor
best original song
best actress tv drama
best actor motion picture drama
best original song
best actress
best animated feature film brave
best original song
best original song
best screenplay quentin tarantino
best motion picture
best supporting actress
best actor motion picture drama
best tv

best foreign film
best original score
best supporting actor
best television series comedy
best original score: mychael danna
best television series
best motion picture
best supporting actress
best motion picture drama
best motion picture, musical
best motion picture comedy
best supporting actress
best motion picture drama
best motion picture drama
best supporting actress
best supporting actor christoph waltz
best motion picture
best motion picture
best picture nominee lincoln
best motion picture (drama
best motion picture, musical
best motion picture, musical
best tv series
best original song: adele
best supporting actor
best movie comedy
best original song
best motion picture drama
best screenplay django unchained
best television series comedy
best original song
best original song
best actor motion picture comedy
best original score mychael danna
best comedy series
best animated feature film brave
best actor motion picture drama
best original song
best foreign language film
best tv se

best supporting actress
best supporting actress
best motion picture drama
best motion picture musical comedy
best original song
best original score
best original score
best tv series actress comedy
best motion picture, musical
best supporting actress
best actor tv comedy
best television miniseries
best director ben affleck
best director award
best original song
best original score mychael danna
best supporting actor
best motion picture comedy
best foreign film
best television series
best director motion picture
best actor best picture
best tv series drama
best director speech
best screenplay quentin tarantino
best peter pan haircut
best motion picture drama
best supporting actor: christoph waltz
best television series
best supporting actr
best original song motion picture award
best supporting actor
best nominee cutaway
best motion picture-drama
best supporting actress
best motion picture drama
best original song
best tv series
best motion picture
best acceptance speech
best original s

In [35]:
new = awardnominees

for lst in new:
    dd = defaultdict( int )
    for word in new[lst]:
        dd[word] += 1
    
    new[lst] = dd

new

defaultdict(<function __main__.<lambda>()>,
            {'Best Motion Picture – Drama': defaultdict(int,
                         {'Chris Terrio': 1,
                          '@BenAffleck': 26,
                          'Jessica Chastain - Zero': 53,
                          'Lewis - Lincoln': 75,
                          'Daniel Day-Lewis': 92,
                          'Jessica Chastain': 98,
                          'George Clooney': 51,
                          'Ben Affleck': 59,
                          'Bill Clinton': 7,
                          '#amcbps': 5,
                          'Affleck': 8,
                          'Motion Picture - Drama': 7,
                          'Daniel Day Lewis': 15,
                          'http://t.co/rNGi3vgg': 1,
                          "Daniel Day Lewis '": 1,
                          '#affleck #': 1,
                          'RT @NBCNews': 5,
                          'Clinton': 1,
                          '#Bert': 1,
       

In [32]:
for each in awardnominees:
    print(each)
    print(len(awardnominees[each]))

Best Motion Picture – Drama
63
Best Motion Picture – Musical or Comedy
29
Best Motion Picture – Foreign Language
0
Best Motion Picture – Animated
0
Best Director – Motion Picture
10
Best Actor in a Motion Picture – Drama
19
Best Actor in a Motion Picture – Musical or Comedy
3
Best Actress in a Motion Picture – Drama
12
Best Actress in a Motion Picture – Musical or Comedy
10
Best Supporting Actor – Motion Picture
7
Best Supporting Actress – Motion Picture
22
Best Screenplay – Motion Picture
11
Best Original Score – Motion Picture
4
Best Original Song – Motion Picture
6
Best Television Series – Drama
31
Best Television Series – Musical or Comedy
15
Best Miniseries or Motion Picture – Television
3
Best Actor in a Television Series – Drama
12
Best Actor in a Television Series – Musical or Comedy
4
Best Actor in a Miniseries or Motion Picture – Television
1
Best Actress in a Television Series – Drama
13
Best Actress in a Television Series – Musical or Comedy
6
Best Actress in a Miniseries o

In [12]:
for each in maybe:
    print(each)
    print(maybe[each])

best screenplay quentin tarantino
34
best supporting actor
68
best motion picture comedy
79
best foreign film
27
best mini-series
9
best original song: adele
25
best television series
44
best director
2
best tv mini series
3
best original song
86
best music score
2
best animated feature film brave
39
best actress winner
4
best director ben affleck
6
best supporting actress
40
best picture nominee lincoln
10
best kate hudson worst lucy lui
2
best television series comedy
19
best tv series comedy
3
best tv series
28
best television series actor drama
28
best beauty look
5
best tv seriesdrama
2
best george hamilton impersonation: leah michelle
2
best movie comedy
15
best original score mychael danna
28
best supporting actor: christoph waltz
8
best motion picture, musical
22
best motion picture
31
best tv actor, comedy
2
best tv series drama
9
best motion picture drama
57
best motion picture (comedy
9
best movie actress
5
best foreign film amour
39
best director award
8
best original score

In [14]:
closeenough("best original score")

TypeError: '<' not supported between instances of 'collections.defaultdict' and 'collections.defaultdict'

In [ ]:
import Levenshtein
print(Levenshtein.distance("Hi Bye", "hi bye"))

In [ ]:
award_list = [dict() for x in range(len(new_award_names))]

for tweet in winners: 
    tweet = tweet[0]
    split = tweet.lower().split()
    
    index = split.index("best")

    award_phrase = " ".join(split[index:])

    name_count = [0]*len(new_award_names)
    for i in range(len(new_award_names)):
#         match_length = len(set(new_award_names[i]).intersection(award_phrase))
#         try Levenshtein distance instead, may be more accurate
        
        name_count[i] = Levenshtein.distance(new_award_names[i], award_phrase)

    award_index = np.argmin(name_count)
    # WHAT ABOUT TIE? Neither? Or Both?
    #print(award_index)
    
    print(name_count)

#     if name_count[award_index] > 1:       
    if "wins" in split:
        person_index = split.index("wins")
    if "won" in split:
        person_index = split.index("won")
    person = split[:index-1]
    person = " ".join(person)
    if person not in award_list[award_index]:
        award_list[award_index][person] = 1
    else:
        award_list[award_index][person] = award_list[award_index][person]+1

print(award_list)

In [ ]:
import operator
i = 0
awards = {}
for award in award_list:
    if award != {}:
        awards[i] = max(award.items(), key=operator.itemgetter(1))[0]
    else:
        awards[i] = ''
    i+=1
print(awards)